In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [2]:
def splitDate(date):
    date = str(date)
    Y = int(date[:4])
    M = int(date[4:6])
    D = int(date[6:])
    return (Y, M, D)

In [3]:
def getWeek(date):
    return datetime(*date).weekday()

In [4]:
read_csv = pd.read_csv("/Users/jooh8/Documents/GitHub/Quant/data/A000660/Index_A000660.txt", sep=" ", low_memory=False)

In [5]:
read_csv["DoW"] = read_csv["날짜"].apply(lambda x: getWeek(splitDate(x)))

In [6]:
weekList = []
preW = 0
week = 0

for W in reversed(read_csv["DoW"]):
    if W > preW:
        week += 1
    weekList.append(week)
    preW = W
    
weekList.reverse()

In [7]:
read_csv["WEEK"] = weekList

In [8]:
for period in range(10):
    
    read_csv["고가_{}".format(period)] = read_csv["고가"].shift(periods=period).fillna(0)

In [9]:
for period in range(10):
    
    read_csv["최고가_{}".format(period)] = read_csv[["고가_{}".format(i) for i in range(period + 1)]].apply(np.max, 1)

In [10]:
for period in range(10):
    
    read_csv["Target_{}".format(period)] = read_csv["시가"] * 1.05 < read_csv["고가_{}".format(period)]

In [11]:
columns = list(read_csv.columns)

In [12]:
momentums = columns[11:-32]
print(momentums)

['A Ratio_20', 'B Ratio_20', 'Aroon_20 Up', 'Down', 'Aroon Osillator_20', 'ATR_14', 'BB-RSI_종가_10', 'BB-RSI_20_2.00 상한', 'BB-RSI 하한', 'BPDL RSI_14', 'BPDL Stochatic_14', 'Chande Momentum Oscillator_10', "Chaikin's Volatility_14,14", 'CompuTrac Volatility_10', '+DI_14', '-DI_14', 'ADX_14', 'ADXR_14', '+DI(simple)_14', '-DI(simple)_14', 'ADX(simple)_14,14', 'ADXR(simple)_14,14,14', 'Energy+_14', 'Energy-_14', 'High Low Envelope_1', 'High Low Oscillator_3', 'Inertia_20,14,10', 'Klinger Oscillator', 'Linear Trend Oscillator_10,20', 'LRS_종가,14', 'Mass Index_20,9', 'Morris Mixed Momentum', 'New BPDL HiLo Index MA_14,7', 'Open Difference_15', 'QStick_14', 'Random Walk Index_15,3', 'RCI_5', 'RCI_9', 'RCI_13', 'RCI_18', 'Relative Volatility Index_단순,14,10', 'Reverse 단기_12', ' 장기_24', 'RSI_종가,14', 'RSI(simple)_종가,14', 'Sigma_종가,20', 'SMI_5,3,3', 'Standard Deviation_14,2.00', 'Standard Error_종가,14', 'Fast %K_5', 'Fast %D_3', 'Slow %K_5,3', 'Slow %D_3', 'Slow(Simple) %K_5,3', 'Slow(Simple) %D_3', 

In [13]:
targets = columns[-10:]
print(targets)

['Target_0', 'Target_1', 'Target_2', 'Target_3', 'Target_4', 'Target_5', 'Target_6', 'Target_7', 'Target_8', 'Target_9']


In [17]:
DATASET = []

for idx in tqdm(range(1, 100)):
    
    X_train = read_csv.loc[(read_csv["WEEK"] > idx+3) & (read_csv["WEEK"] < idx+303), momentums]
    Y_train = read_csv.loc[(read_csv["WEEK"] > idx+3) & (read_csv["WEEK"] < idx+303), targets]
    
    TESTSET = []

    X_test = read_csv.loc[(read_csv["WEEK"] == idx+3), momentums]
    Y_test = read_csv.loc[(read_csv["WEEK"] == idx+3), targets]
    
    TESTSET.append([X_test, Y_test])
    
    X_test = read_csv.loc[(read_csv["WEEK"] == idx+2), momentums]
    Y_test = read_csv.loc[(read_csv["WEEK"] == idx+2), targets]
    
    TESTSET.append([X_test, Y_test])
    
    X_test = read_csv.loc[(read_csv["WEEK"] == idx+1), momentums]
    Y_test = read_csv.loc[(read_csv["WEEK"] == idx+1), targets]
    
    TESTSET.append([X_test, Y_test])
    
    X_test = read_csv.loc[(read_csv["WEEK"] == idx), momentums]
    Y_test = read_csv.loc[(read_csv["WEEK"] == idx), targets]
    
    TESTSET.append([X_test, Y_test])
    
    DATASET.append([[X_train, Y_train], TESTSET])

100%|███████████████████████████████████████████████████████| 99/99 [00:01<00:00, 52.48it/s]


In [33]:
DATASET[0][1][3][1]

,Target_0,Target_1,Target_2,Target_3,Target_4,Target_5,Target_6,Target_7,Target_8,Target_9
5612,False,False,True,True,True,True,True,True,True,True
5613,False,False,True,True,True,True,True,True,True,True
5614,False,False,False,False,True,True,True,True,True,True
5615,False,False,False,False,False,True,True,True,True,True
5616,False,False,False,False,False,True,True,True,True,True


In [21]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
import numpy as np

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)

forest = RandomForestClassifier(n_estimators=500, random_state=1, max_depth=5)

multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

multi_target_forest.fit(scaler.transform(X_train), Y_train)

predict = multi_target_forest.predict(scaler.transform(X_train))

# corr = predict == Y_train

# corr.apply(np.mean, 0)

# np.mean(corr.apply(np.mean, 0))

predict = multi_target_forest.predict(scaler.transform(X_test))

# corr = predict == Y_test

# corr.apply(np.mean, 0)

# np.mean(corr.apply(np.mean, 0))

cm = confusion_matrix(np.array(Y_test["Target_9"]), predict[:, 9])

Sensitivity = cm[1, 1] / np.sum(cm[:, 1])
Accuracy = np.sum(cm.diagonal()) / np.sum(cm)

In [ ]:
Sensitivity, Accuracy